<a href="https://colab.research.google.com/github/nvwa0318/Supervise-learning/blob/main/supervised_learning_xii2_hyper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

from sklearn import svm, datasets
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn import metrics

columns = ["sex","length","diam","height","whole","shucked","viscera","shell","age"]
df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data",names=columns)

y = df.age
X=df.drop(columns=['age'])
num_cols = X.select_dtypes(include=np.number).columns
cat_cols = X.select_dtypes(include=['object']).columns
#create some missing values
for i in range(1000):
    X.loc[np.random.choice(X.index),np.random.choice(X.columns)] = np.nan

x_train, x_test, y_train, y_test = train_test_split(X,y, random_state=0, test_size=0.25)

cat_vals = Pipeline([("imputer",SimpleImputer(strategy='most_frequent')), ("ohe",OneHotEncoder(sparse=False, drop='first'))])
num_vals = Pipeline([("imputer",SimpleImputer(strategy='mean')), ("scale",StandardScaler())])

preprocess = ColumnTransformer(
    transformers=[
        ("cat_process", cat_vals, cat_cols),
        ("num_process", num_vals, num_cols)
    ]
)
#Create a pipeline with preprocess and a linear regression model
pipeline = Pipeline([("preprocess",preprocess), 
                     ("regr",LinearRegression())])
#Very simple parameter grid, with and without the intercept
param_grid = {
    "regr__fit_intercept": [True,False]
}

# 1. Update the search_space dictionary to include values for alpha in lasso and ridge regression models. Use np.logspace(-4,2,10).
search_space = [{'regr': [LinearRegression()]}, # Actual Estimator
                {'regr':[Ridge()],
                     'regr__alpha': np.logspace(-4, 2, 10)},
                {'regr':[Lasso()],
                     'regr__alpha': np.logspace(-4, 2, 10)}]


# 2.  Fit the GridSearchCV on the training data and print the best estimator and score from the search.
gs = GridSearchCV(pipeline, search_space, scoring='neg_mean_squared_error', cv=5)
gs.fit(x_train, y_train)
print('Best Estimator:')
print(gs.best_estimator_)
print('Best NMSE:')
print(gs.best_score_)

#3. Save the best estimator and print it
best_model = gs.best_estimator_
print('The regression model is:')
print(best_model.named_steps['regr'])
print('The hyperparameters of the regression model are:')
print(best_model.named_steps['regr'].get_params())

#4. Access the hyperparameters of the categorical preprocessing step

print('The hyperparameters of the imputer are:')
print(best_model.named_steps['preprocess'].named_transformers_['cat_process'].named_steps['imputer'].get_params())